In [1]:
from sklearn.model_selection import KFold
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
import function
import matplotlib.pyplot as plt
import math
import tqdm
import optuna

In [2]:
p_1=50
p_2=50
n=150
r_J_or=10
r_1_or=10
r_2_or=10
r_prop=1.0
w_J=1.0
w_1=1.0
w_2=1.0
X1_erro=0.2
X2_erro=0.2
y_erro=0.0001

In [3]:
X_1_al,X_2_al,y_al=function.gene_data(p_1,p_2,n,r_J_or,r_1_or,r_2_or,r_prop,w_J,w_1,w_2,X1_erro,X2_erro,y_erro)

In [4]:
X_1_original=X_1_al[:,:100]
X_2_original=X_2_al[:,:100]
y_original=y_al[:,:100]

X_1_test=X_1_al[:,100:150]
X_2_test=X_2_al[:,100:150]
y_test=y_al[:,100:150]

In [5]:
df_tra=np.row_stack((X_1_original,X_2_original,y_original))
df_tes=np.row_stack((X_1_test,X_2_test,y_test))

In [6]:
U_J,sigma_J,VT_J=np.linalg.svd(X_1_original,full_matrices=False)



In [44]:
diag_sigma_J=np.diag(sigma_J**2)
diag_sigma_J

array([[2.05545192e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.53836278e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.78422113e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.18854473e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.93715591e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 8.13380352e-06]])

In [43]:
pclasso_sigma_J=[]
for i in range(len(sigma_J)):
    a_i=sigma_J[0]**2-sigma_J[i]**2
    pclasso_sigma_J.append(a_i)
pclasso_sigma_J_flo=np.float_(pclasso_sigma_J)
diag_pcLasso_sigma_J=np.diag(pclasso_sigma_J_flo)
diag_pcLasso_sigma_J

array([[   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        , 1801.61564255,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        , 1877.02980785, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ..., 2055.44003543,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
        2055.44898372,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 2055.45191274]])

In [27]:
eta=0.5
times=100
r_J=10
r_1=10
r_2=10
threshold=100.0
gamma_1=1.0
gamma_21=1.0
gamma_22=1.0
X_1_or=X_1_original
X_2_or=X_2_original
y_or=y_original

In [22]:
times=times
erro_lis=[]
erro_best=1000.0
threshold=threshold
p_1=X_1_or.shape[0]
p_2=X_2_or.shape[0]
n=X_1_or.shape[1]
X_1=math.sqrt(eta)*X_1_or
X_2=math.sqrt(eta)*X_2_or
y=math.sqrt(1-eta)*y_or

In [23]:
U_1=U_1_best=math.sqrt(eta)*np.random.uniform(low=-0.1, high=0.1, size=(p_1,r_J))
U_2=U_2_best=math.sqrt(eta)*np.random.uniform(low=-0.1, high=0.1, size=(p_2,r_J))
W_1=W_1_best=math.sqrt(eta)*np.random.uniform(low=-0.1, high=0.1, size=(p_1,r_1))
W_2=W_2_best=math.sqrt(eta)*np.random.uniform(low=-0.1, high=0.1, size=(p_2,r_2))
theta_1=theta_1_best=math.sqrt(1-eta)*np.random.uniform(low=-0.1, high=0.1, size=int(r_J))
theta_21=theta_21_best=math.sqrt(1-eta)*np.random.uniform(low=-0.1, high=0.1, size=int(r_1))
theta_22=theta_22_best=math.sqrt(1-eta)*np.random.uniform(low=-0.1, high=0.1, size=int(r_2))
S_J=S_J_best=np.random.uniform(low=-0.1, high=0.1, size=(r_J,n))
S_1=S_1_best=np.random.uniform(low=-0.1, high=0.1, size=(r_1,n))
S_2=S_2_best=np.random.uniform(low=-0.1, high=0.1, size=(r_2,n))
hat_X_y_best=np.random.uniform(low=-0.1, high=0.1, size=(p_1+p_2+1,n))

In [24]:
zeros_1=np.zeros((p_2, r_1))
zeros_2=np.zeros((p_1, r_2))

In [25]:
U_theta_1=np.row_stack((U_1,U_2,theta_1)) #1
X_y=np.row_stack((X_1,X_2,y)) #2
W_1_S_1=W_1.dot(S_1)
W_2_S_2=W_2.dot(S_2)
theta_2i_S_i=theta_21.dot(S_1)+theta_22.dot(S_2)
W_theta_2i_S_i=np.row_stack((W_1_S_1,W_2_S_2,theta_2i_S_i)) #3
S_J=U_theta_1.T.dot(X_y-W_theta_2i_S_i) #1,2,3からS_Jを更新

In [67]:
#U_1,U_2,theta_1を更新
X_y_joint=X_y-W_theta_2i_S_i
U_J,sigma_J,VT_J=np.linalg.svd(X_y_joint,full_matrices=False) #特異値分解を行う
U_1=U_J[:p_1,:r_J]
U_2=U_J[p_1:p_1+p_2,:r_J]
#提案手法の更新 
pcLasso_sigma_J=[]
for i in range(len(sigma_J)):
    a_i=sigma_J[0]**2-sigma_J[i]**2
    pcLasso_sigma_J.append(a_i)
sigma_J_final=[1/(sigma_J[i]**2+(gamma_1/2)*pcLasso_sigma_J[i]) for i in range(len(sigma_J))]
diag_sigma_J_final=np.diag(sigma_J_final)
diag_sigma_J_final_r_J=diag_sigma_J_final[:r_J,:r_J]
theta_1=(y-(theta_21.dot(S_1)+theta_22.dot(S_2))).dot(S_J.T).dot(diag_sigma_J_final_r_J)
U_theta_1=np.row_stack((U_1,U_2,theta_1))
#正規化
U_theta_1=U_theta_1/np.linalg.norm(U_theta_1)
U_1=U_theta_1[:p_1,:]
U_2=U_theta_1[p_1:p_1+p_2,:]
theta_1=U_theta_1[p_1+p_2:,:]
U_theta_1=np.row_stack((U_1,U_2,theta_1))

In [ ]:
#W_1,theta_21を更新
##特異値分解
U_I_1,sigma_1,VT_1=np.linalg.svd((X_1_y_theta_22S_2-U_1_S_J_theta_1_S_J).dot(P_SJ_C),full_matrices=False) #特異値分解を行う
W_1=U_I_1[:p_1,:r_1]
zeros_1=np.zeros((p_2, r_1))
##提案手法の更新
pcLasso_sigma_1=[]
for i in range(len(sigma_1)):
    a_i=sigma_1[0]**2-sigma_1[i]**2
    pcLasso_sigma_1.append(a_i)
sigma_1_final=[1/(sigma_1[i]**2+(gamma_21/2)*pcLasso_sigma_1[i]) for i in range(len(sigma_1))]
diag_sigma_1_final=np.diag(sigma_1_final)
diag_sigma_1_final_r_1=diag_sigma_J_final[:r_1,:r_1]
theta_21=(y-(theta_21.dot(S_1)+theta_22.dot(S_2))).dot(S_1.T).dot(diag_sigma_1_final_r_1)
##正規化
W_1_theta_21=np.row_stack((W_1,zeros_1,theta_21))
W_1_theta_21=W_1_theta_21/np.linalg.norm(W_1_theta_21)
W_1=W_1_theta_21[:p_1,:]
theta_21=W_1_theta_21[p_1+p_2:,:]
W_1_theta_21=np.row_stack((W_1,zeros_1,theta_21))

In [ ]:
#W_1,theta_21を更新
##特異値分解
U_I_2,sigma_2,VT_2=np.linalg.svd((X_2_y_theta_21S_1-U_2_S_J_theta_1_S_J).dot(P_SJ_C),full_matrices=False) #特異値分解を行う
zeros_2=np.zeros((p_1, r_2))
W_2=U_I_2[:p_2,:r_2]
##提案手法の更新
pcLasso_sigma_2=[]
for i in range(len(sigma_2)):
    a_i=sigma_2[0]**2-sigma_2[i]**2
    pcLasso_sigma_2.append(a_i)
sigma_2_final=[1/(sigma_2[i]**2+(gamma_22/2)*pcLasso_sigma_2[i]) for i in range(len(sigma_2))]
diag_sigma_2_final=np.diag(sigma_2_final)
diag_sigma_2_final_r_2=diag_sigma_2_final[:r_2,:r_2]
theta_22=(y-(theta_1.dot(S_J)+theta_21.dot(S_1))).dot(S_2.T).dot(diag_sigma_2_final_r_2)
##正規化
W_2_theta_22=np.row_stack((zeros_2,W_2,theta_22))
W_2_theta_22=W_2_theta_22/np.linalg.norm(W_2_theta_22)
W_2=W_2_theta_22[p_1:p_1+p_2,:]
theta_22=W_2_theta_22[p_1+p_2:,:]
W_2_theta_22=np.row_stack((zeros_2,W_2,theta_22))

In [71]:
U_theta_1_norm=U_theta_1/np.linalg.norm(U_theta_1)

In [72]:
U_theta_1.T.dot(U_theta_1)

array([[ 9.99994143e-01, -4.24646633e-04, -4.53305028e-04,
         6.25061382e-04,  3.55327522e-04,  2.05748972e-04,
         4.26336081e-04,  5.35880736e-04,  8.87503499e-05,
         3.16021877e-04],
       [-4.24646633e-04,  9.69397140e-01, -3.26923573e-02,
         4.51689047e-02,  2.56619949e-02,  1.48757780e-02,
         3.07634101e-02,  3.87855208e-02,  6.36578763e-03,
         2.28491737e-02],
       [-4.53305028e-04, -3.26923573e-02,  9.65078656e-01,
         4.82368037e-02,  2.74069502e-02,  1.58851396e-02,
         3.28587724e-02,  4.14118524e-02,  6.80439135e-03,
         2.43994737e-02],
       [ 6.25061382e-04,  4.51689047e-02,  4.82368037e-02,
         9.33414008e-01, -3.78398154e-02, -2.19240809e-02,
        -4.53799835e-02, -5.71352383e-02, -9.41583203e-03,
        -3.36749601e-02],
       [ 3.55327522e-04,  2.56619949e-02,  2.74069502e-02,
        -3.78398154e-02,  9.78497436e-01, -1.24597539e-02,
        -2.57850882e-02, -3.24740818e-02, -5.34698983e-03,
        -1.

In [59]:
theta_1=diag_sigma_J_final.dot((y-(theta_21.dot(S_1)+theta_22.dot(S_2))).dot(S_J.T))

ValueError: shapes (100,100) and (1,10) not aligned: 100 (dim 1) != 1 (dim 0)